# Meta learner

## Prepare data set: split, scaler, and create tensorflow data sets


In [ ]:
import os

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd
import tensorflow as tf
tf.random.set_seed(123)

# Read the data set
# df = pd.read_pickle('pkls/dataset.pkl')

df = df.sample(frac=1, random_state=123)

df['scaled_value'] = df['val']
skf = StratifiedKFold(n_splits=5)

props = df.prop.unique().tolist()

datasets = []
def get_dataset(df):
    # return TF data set
    fps_mix = np.stack(df.fps_mix.values).astype(np.float32)
    selector = np.stack(df.dummy.values).astype(np.float32)   
    target = df.scaled_value.astype(np.float32)[:, np.newaxis]
    dataset = tf.data.Dataset.from_tensor_slices(({'sel': selector, 'fps_mix': fps_mix, 'prop': df.prop}, target))

    dataset = dataset.cache().batch(200).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


training_df, test_df = train_test_split(df, test_size=0.2, stratify=df.prop, random_state=123)
training_df, test_df = training_df.copy(), test_df.copy()

for train_index, val_index in skf.split(training_df, training_df.prop):
    # iterte over 5 splits
    train_df = training_df.iloc[train_index].copy()
    val_df = training_df.iloc[val_index].copy()
    
    # scale target values
    property_scaler = {}
    for prop in props:
        property_scaler[prop] = MinMaxScaler()

        # train
        cond = train_df[train_df.prop == prop].index
        train_df.loc[cond, ['scaled_value']] = property_scaler[prop].fit_transform(train_df.loc[cond, ['scaled_value']])
        
        # val
        cond = val_df[val_df.prop == prop].index
        val_df.loc[cond, ['scaled_value']] = property_scaler[prop].transform(val_df.loc[cond, ['scaled_value']])

    datasets.append({'train': get_dataset(train_df), 'val': get_dataset(val_df), 'property_scaler': property_scaler})

# Create final dataset for meta learner
property_scaler_final = {}
for prop in props:
    property_scaler_final[prop] = MinMaxScaler()
    
   # train
    cond = training_df[training_df.prop == prop].index
    training_df.loc[cond, ['scaled_value']] = property_scaler_final[prop].fit_transform(training_df.loc[cond, ['scaled_value']])

    # val
    cond = test_df[test_df.prop == prop].index
    test_df.loc[cond, ['scaled_value']] = property_scaler_final[prop].transform(test_df.loc[cond, ['scaled_value']])
    
datasets_final = {'train': get_dataset(training_df), 'test': get_dataset(test_df), 'property_scaler': property_scaler_final}


## Definition of the meta learner

In [ ]:
import tensorflow.keras as tfk
import tensorflow as tf
from datetime import datetime 
from tensorflow.python.keras.engine import data_adapter
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband, RandomSearch



class MetaModel(tfk.Model):
    """Meta learner class"""

    def __init__(self, hp):
        super().__init__()
        self.base_models = []
        for num in range(5):
            # load all cross-validation models
            model = tf.keras.models.load_model(f'models/fp/{num}')
            model.trainable = False
            self.base_models.append(model)
        

        self.my_layers = []
        for i in range(hp.Int('num_layers', 1, 2)): 
            new_step = [               
            tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=64,
                                            max_value=544,
                                            step=64),),
            
            tf.keras.layers.PReLU(),
            tf.keras.layers.Dropout(hp.Float(
                'dropout_' + str(i),
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )),]
            self.my_layers.append(new_step)
            
        self.my_layers.append([tf.keras.layers.Dense(1)])

    def call(self, inputs, training=None): 
        
        # drop prop if there
        if 'prop' in inputs:
            del inputs['prop']

        x = []
        for base in self.base_models:
            if training:
                res = base.call(inputs, training)
            else:
                res = base.call(inputs)
            x.append(res)
        x = tf.concat(x, -1)
        
        for num, layer_step in enumerate(self.my_layers):
            for layer in layer_step:
                x = layer(x)

        return x
    
    def predict_step(self, data):
        data = data_adapter.expand_1d(data)
        x, _, _ = data_adapter.unpack_x_y_sample_weight(data)

        # drop prop here
        prop = x['prop']
        del x['prop']
        return self(x, training=True), data[-1], prop

        

def build_model(hp):
    # returns the compiled TF model
    model = MetaModel(hp)
    opt = tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-3]))
    opt = tfa.optimizers.SWA(opt)

    model.compile(
        optimizer=opt,
        loss='mse',)
    return model

## Train and optimize the meta learner

In [ ]:
import IPython
from sklearn.metrics import mean_squared_error, r2_score
results, property_metric = [], []


tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=300,
    seed=10,
    directory=f'hyperparameter_search_meta_learner',
    project_name='fold_0',
    )

reduce_lr = tfk.callbacks.ReduceLROnPlateau(
    factor=0.9,
    monitor="val_loss",
    verbose=1,
)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=40)

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

# Create an instance of the model and optimize the hyperspace
tuner.search(datasets_final['test'],
            epochs=300,
            validation_data=datasets_final['train'],
            callbacks=[earlystop, reduce_lr, ClearTrainingOutput()],
            verbose=0
            )

# Post processing
best_values = tuner.get_best_hyperparameters()[0].values
best_model = tuner.get_best_models(1)[0]

# predict
res = np.concatenate(best_model.predict(datasets_final['train']), -1)

# save for deployment
best_model.save(f'models/meta_model/', include_optimizer=False)


_df = pd.DataFrame(res, columns=['pred', 'target', 'prop'])
_df['prop'] = _df.prop.apply(lambda x: x.decode('utf-8'))
props = _df.prop.unique()

property_scaler = datasets_final['property_scaler']
for prop in props:

    cond = _df[_df.prop == prop].index
    rmse_scaled = mean_squared_error(_df.loc[cond, ['target']], _df.loc[cond, ['pred']], squared=False)
    r2_scaled = r2_score(_df.loc[cond, ['target']], _df.loc[cond, ['pred']])
    
    _df.loc[cond, ['pred']] = property_scaler[prop].inverse_transform(_df.loc[cond, ['pred']])
    _df.loc[cond, ['target']] = property_scaler[prop].inverse_transform(_df.loc[cond, ['target']])
    
    rmse = mean_squared_error(_df.loc[cond, ['target']], _df.loc[cond, ['pred']], squared=False)
    r2 = r2_score(_df.loc[cond, ['target']], _df.loc[cond, ['pred']])
    property_metric.append({'name': f'ensemble_{what}', 'prop': prop, 'rmse': rmse, 'r2':r2, 'rmse_scaled': rmse_scaled, 'r2_scaled': r2_scaled})
    
# Not scaled back
rmse = mean_squared_error(res[:,0], res[:,1], squared=False)
r2 = r2_score(res[:,0], res[:,1])

results.append({'name': f'ensemble_{what}','r2': r2, 'rmse':rmse})

pd.DataFrame(results)    
